---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [209]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [210]:
import re
import sys
def dateExtractor(inputStr:str)->list:
    """
    Given a string, extract and return the list of dates
    6/1998
    """
    patterns = ['\d{1,2}[/-]\d{1,2}[/-]\d{2,4}',
               '\d{1,2}[ ]?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ ]\d{4}',
                '\d{1,2}[ ]?(?:January|February|March|April|May|June|July|August|September|October|November|December)[ ]\d{4}',
                '(?:January|February|March|April|May|June|July|August|September|October|November|December)[ ]\d{1,2}[,][ ]\d{2,4}',
                '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ ]\d{1,2}[,][ ]\d{2,4}',
                '(?:January|February|March|April|May|June|July|August|September|October|November|December)[.][ ]\d{1,2}[,][ ]\d{2,4}',
                '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ ]\d{1,2}[ ]\d{2,4}',
                '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.][ ]\d{1,2}[,][ ]\d{2,4}',
                '(?:January|February|March|April|May|June|July|August|September|October|November|December)[ ]\d{1,2}[ ]\d{2,4}',
                '(?:January|February|March|April|May|June|July|August|September|October|November|December)[ ]\d{2,4}',
                '(?:January|February|March|April|May|June|July|August|September|October|November|December)[,][ ]\d{2,4}',
                '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ ]\d{2,4}',
                '(?:Janaury|February|March|April|May|June|July|August|September|October|November|Decemeber)[ ]\d{2,4}',
                '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[,][ ]\d{2,4}',
                '\d{1,2}[/]\d{2,4}',
                '\d{4}'
               ]
    
    for pattern in patterns:
        dateStr = re.findall(pattern, inputStr)
        if len(dateStr)>0:
            return dateStr
        
    sys.exit(f'No date found for input: {inputStr}')

    

In [211]:
def processYear(inpYear:str):
    """
    Check if the year
    """
    if len(inpYear)==2:
        # assume a 1900 based year
        return f'19{inpYear}'
    elif len(inpYear)==4:
        return inpYear

def processMonth(inpMonth):
    """
    This will process months to validate they are returned in mm format
    Will handle months of Jan, Feb format
    Will handle months of January, February format
    
    Returns
    -------
    month:str
        2 digit month: ie 01, 02, 03, etc
    """
    if len(inpMonth) <=2:
        return inpMonth
    
    monthDict = {
        "jan": "01",
        "january" :"01",
        "january,":"01",
        "janaury":"01",
        "feb": "02",
        "february": "02",
        "february,":"02",
        "mar":"03",
        "march":"03",
        "march,":"03",
        "mar.":"03",
        "mar,":"03",
        "apr":"04",
        "apr,":"04",
        "april":"04",
        "may":"05",
        "may":"05",
        "may,":"05",
        "june":"06",
        "jun":"06",
        "june,":"06",
        "jul":"07",
        "july":"07",
        "july,":"07",
        "aug":"08",
        "august":"08",
        "sep":"09",
        "september":"09",
        "sep.":"09",
        "september.":"09",
        "oct":"10",
        "october":"10",
        "october.":"10",
        "nov":"11",
        "november":"11",
        "dec":"12",
        "december":"12",
        "decemeber":"12"
    }
    
    inpMonth = inpMonth.lower()
    
    digitizedMonth = monthDict[inpMonth]
    
    return digitizedMonth
    
    

def createProperYear(inputDate:list)->str:
    """
    This will format all inputs to mm/dd/yyyy format for a later ordering function
    
    If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
    Watch out for potential typos as this is a raw, real-life derived dataset.
    
    Returns
    ------
    date:str
        date in mm/dd/yyyy format
    """
    yearOnly=False
    
    # check list length
    if len(inputDate)>1:
        return "No handling yet for lists with len > 1"
    elif len(inputDate)==1 and len(inputDate[0])==4:
        dateList=inputDate
        yearOnly=True
    
    inputDate=inputDate[0]
    
    # check data format
    if yearOnly==True:
        # assume it is the first of January that year
        day = '1'
        month = '1'
        year = dateList
        
        month=processMonth(month)
    elif '/' in inputDate:
        dateList = inputDate.split('/')
    elif '-' in inputDate:
        dateList = inputDate.split('-')
    elif ' ' in inputDate:
        dateList = inputDate.split(' ')
    else:
        sys.exit(f'Handling for input: {inputDate} not created')
    
    if len(dateList)==3:
        month=dateList[0]
        day=dateList[1]
        year=dateList[2]
        
        year=processYear(year)
        month=processMonth(month)
    
    elif len(dateList)==2:
        # assume this is mm/yyyy or mm/yy handling
        day = 1
        month = dateList[0]
        year=dateList[1]
        
        month=processMonth(month)
        
    elif len(dateList)==1:
        # assume it is the first of January that year
        day = '1'
        month = '1'
        year = dateList[0]
        
        month=processMonth(month)
    else:
        return "No handling yet for / or - split with more than 3 results"
    
    returnDate = f'{month}/{day}/{year}'
    return returnDate
    
def getSortedList(unsortedList:list)->list:
    """
    Given a list, sort the list in chronological order and return the sorted list
    
    Parameters
    ----------
    unsortedList:list
        Unsorted list in form mm/dd/yyyy
    
    Returns
    -------
    sortedList:list
        Sorted list in chronological order
    """
    sortedList = []
    
    for record in unsortedList:
        if len(sortedList)==0:
            sortedList.append(record)
        else:
            unsortedRecordList = record.split('/')
            day = unsortedRecordList[0]
            month = unsortedRecordList[1]
            year = unsortedRecordList[2]
            
            # compare year
            
    
            # compare month

            # compare day
            
        
    

In [212]:
def date_sorter():
    true_df = pd.DataFrame(df, columns=['original_text'])
    true_df['text'] = true_df['original_text']
    for location,record in enumerate(true_df['original_text']):
        result = dateExtractor(record)
        formatted=createProperYear(result)
        true_df['text'].iloc[location]=formatted
    
        
    #myList = df.tolist()
    
    #real_df = pd.DataFrame(myList, columns=['text'])
    #real_df['original_text'] = real_df['text']
    

    true_df[['month', 'day', 'year']] = true_df['text'].str.split('/',3,expand=True)
    year_sort_df = true_df.sort_values(by=['year', 'month', 'day'], ascending=True)
    year_sort_df.reset_index(inplace=True)
    year_sort_df['dateRank'] = year_sort_df.index
    finalSeries = year_sort_df['dateRank']
    year_sort_df.set_index('index', inplace=True)
    year_sort_df['original_index']=year_sort_df.index
    year_sort_df.set_index('dateRank', inplace=True)
    
    #return sorted series
    finalSeries = year_sort_df['original_index']
    return year_sort_df


Input is: ['03/25/93']
Input:['03/25/93']\Output:03/25/1993
Input is: ['6/18/85']
Input:['6/18/85']\Output:6/18/1985
Input is: ['7/8/71']
Input:['7/8/71']\Output:7/8/1971
Input is: ['9/27/75']
Input:['9/27/75']\Output:9/27/1975
Input is: ['2/6/96']
Input:['2/6/96']\Output:2/6/1996
Input is: ['7/06/79']
Input:['7/06/79']\Output:7/06/1979
Input is: ['5/18/78']
Input:['5/18/78']\Output:5/18/1978
Input is: ['10/24/89']
Input:['10/24/89']\Output:10/24/1989
Input is: ['3/7/86']
Input:['3/7/86']\Output:3/7/1986
Input is: ['4/10/71']
Input:['4/10/71']\Output:4/10/1971
Input is: ['5/11/85']
Input:['5/11/85']\Output:5/11/1985
Input is: ['4/09/75']
Input:['4/09/75']\Output:4/09/1975
Input is: ['8/01/98']
Input:['8/01/98']\Output:8/01/1998
Input is: ['1/26/72']
Input:['1/26/72']\Output:1/26/1972
Input is: ['5/24/1990']
Input:['5/24/1990']\Output:5/24/1990
Input is: ['1/25/2011']
Input:['1/25/2011']\Output:1/25/2011
Input is: ['4/12/82']
Input:['4/12/82']\Output:4/12/1982
Input is: ['10/13/1976']
I

Input:['21 Oct 2007']\Output:21/Oct/2007
Input is: ['19 Oct 2016']
Input:['19 Oct 2016']\Output:19/Oct/2016
Input is: ['05 Mar 1974']
Input:['05 Mar 1974']\Output:05/Mar/1974
Input is: ['29 Jan 1994']
Input:['29 Jan 1994']\Output:29/Jan/1994
Input is: ['21 Oct 1978']
Input:['21 Oct 1978']\Output:21/Oct/1978
Input is: ['18 August 1975']
Input:['18 August 1975']\Output:18/August/1975
Input is: ['11 Nov 1996']
Input:['11 Nov 1996']\Output:11/Nov/1996
Input is: ['01 Oct 1979']
Input:['01 Oct 1979']\Output:01/Oct/1979
Input is: ['13 Oct 1986']
Input:['13 Oct 1986']\Output:13/Oct/1986
Input is: ['21 Oct 1995']
Input:['21 Oct 1995']\Output:21/Oct/1995
Input is: ['24 Jan 2011']
Input:['24 Jan 2011']\Output:24/Jan/2011
Input is: ['04 Oct 1972']
Input:['04 Oct 1972']\Output:04/Oct/1972
Input is: ['23 Aug 1993']
Input:['23 Aug 1993']\Output:23/Aug/1993
Input is: ['18 Oct 2006']
Input:['18 Oct 2006']\Output:18/Oct/2006
Input is: ['04 Dec 1988']
Input:['04 Dec 1988']\Output:04/Dec/1988
Input is: ['

In [213]:
result = date_sorter()

In [214]:
result

['03/25/1993',
 '6/18/1985',
 '7/8/1971',
 '9/27/1975',
 '2/6/1996',
 '7/06/1979',
 '5/18/1978',
 '10/24/1989',
 '3/7/1986',
 '4/10/1971',
 '5/11/1985',
 '4/09/1975',
 '8/01/1998',
 '1/26/1972',
 '5/24/1990',
 '1/25/2011',
 '4/12/1982',
 '10/13/1976',
 '4/24/1998',
 '5/21/1977',
 '7/21/1998',
 '10/21/1979',
 '3/03/1990',
 '2/11/1976',
 '07/25/1984',
 '4/13/1982',
 '9/22/1989',
 '9/02/1976',
 '9/12/1971',
 '10/24/1986',
 '03/31/1985',
 '7/20/1972',
 '4/12/1987',
 '06/20/1991',
 '5/12/2012',
 '3/15/1983',
 '2/14/1973',
 '5/24/1988',
 '7/27/1986',
 '1/14/1981',
 '7/29/1975',
 '6/24/1987',
 '8/14/1994',
 '4/13/2002',
 '8/16/1982',
 '2/15/1998',
 '7/15/1991',
 '06/12/1994',
 '9/17/1984',
 '2/28/1975',
 '11/22/1975',
 '5/24/1991',
 '6/13/1992',
 '7/11/1971',
 '12/26/1986',
 '10/11/1987',
 '3/14/1995',
 '12/01/1973',
 '12/5/2010',
 '08/20/1982',
 '7/24/1995',
 '8/06/1983',
 '02/22/1992',
 '6/28/1987',
 '07/29/1994',
 '08/11/1978',
 '10/29/1991',
 '7/6/1991',
 '1/21/1987',
 '11/3/1985',
 '7/04